[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/Pesquisa-Operacional-III-A/blob/main/9_PNL.ipynb)

## **Pesquisa Operacional III-A**

**Professores:**
- Diogo Ferreira de Lima Silva (TEP-UFF)
- Marcos Costa Roboredo (TEP-UFF)


# Programação Não Linear

Para resolver os problemas de PNL em uma ferramenta computacional, precisamos de uma linguagem de modelagem matemática e um solver. 

Vimos anteriormente alguns exemplos de modelagem e Pyomo com posterior uso do solver Gurobi. 

A boa notícia é que podemos continuar usando o mesmo framework de modelagem. A única diferença é que precisaremos de um resolvedor diferente.

## Instalação das bibliotecas

Inicialmente, se estiver trabalhando no *Google Colab*, você precisará instalar a biblioteca Pyomo. 

Neste caso, execute o bloco de código abaixo.

In [ ]:
!pip install pyomo # Instala o Pyomo no ambiente de execução

Em seguida, para a instalação do *solver*. Nesta aula, utilizaremo o **ipopt** (Interior Point OPTimizer).

Para mais informações, acesse: https://github.com/coin-or/Ipopt

Para a instalção, execute as duas células abaixo:

In [ ]:
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"

In [ ]:
!unzip -o -q ipopt-linux64

## Introdução -  Modelando um problema de PNL

Precisamos importar a biblioteca:

In [ ]:
import pyomo.environ as pe 

### Criação do Modelo

Para a criação do modelo, devemos inicialmente instanciá-lo e atribuí-lo a uma variável. 

Chamaremos nosso modelo de M.

In [ ]:
# MODELO
M = pe.ConcreteModel() 

### Variáveis

Vamos criar as variáveis do nosso modelo. Para isso, usamos o método $pe.Var$. 

Podemos inclusive indicar já neste moomento um domínio para as variáveis.

Neste caso, criaremos indicando que são números reais não negativos.

Seria possível ainda indicar um limitante superior, caso necessário. Por exemplo, poderiamos definir uma variável entre 0 e 3 com o parâmetro: $bounds=(0, 6)$,

In [ ]:
x = M.x = pe.Var(domain= pe.NonNegativeReals) # Definindo a variável x
y = M.y = pe.Var(domain= pe.NonNegativeReals) # Definindo a variável y

### Função Objetivo

A função objetivo é definida com o método pe.Objective. Devemos indicar se o problema é de maximização ou minimização, além da expressão da FO.

In [ ]:
z = M.z = pe.Objective(expr= -3 * x**2 + y, sense= pe.maximize) # Definindo a função objetivo

### Restrições

As restrições do modelo são incluídas com o método pe.Constraint. Vejamos o exemplo abaixo para uma função chamada de $R1$:

In [ ]:
# RESTRIÇÃO
M.R1 = pe.Constraint(expr= x**2 + y**2 <= 1) 


### Resolução

O modelo de PNL deve ser resolvido com um *solver* apropriado. O Pyomo é apenas um framework que apresenta uma linguagem para modelagem. No entanto, ele não resolve os problemas.

Por outro lado, o Pyomo possui integração com uma vasta quantidade de solvers. Dentre eles, está o **ipopt**, um solver apropriado para problemas de Programação Não Linear em que a função objetivo e as restrições são compostas por equações duas vezes diferenciáveis.


In [ ]:
pe.SolverFactory('ipopt').solve(M)

# Imprimindo o resultado na tela
print(f"z= {pe.value(M.z)}\nx= {pe.value(M.x)}\ny= {pe.value(M.y)}")

## Um Problema Irrestrito com uma variável

$$\max{f(x_1, x_2)} = 12x - 3x^4 - 2x^6 $$


In [ ]:
M = pe.ConcreteModel() 
x = M.x = pe.Var()
M.f = pe.Objective(expr=  12*x + 3 * x**4 + 2*x**6, sense= pe.maximize) 
pe.SolverFactory('ipopt').solve(M)

# Imprimindo o resultado na tela
print(f"f(x[1], x[2])= {pe.value(M.z)}\nx= {pe.value(M.x)}")

## Um Problema Irrestrito com 2 variáveis

$$\max{f(x_1, x_2)} = 2x_1x_2 + 2x_2 - x_1^2 - 2x_2^2$$


In [ ]:
M = pe.ConcreteModel() 
M.J = pe.Set([1,2])
x = M.x = pe.Var(M.J)
M.f = pe.Objective(expr=  2*x[1]*x[2] + 2*x[2] - x[1]**2 - 2*x[2]**2, sense= pe.maximize) 
pe.SolverFactory('ipopt').solve(M)

# Imprimindo o resultado na tela
print(f"f(x[1], x[2])= {pe.value(M.z)}\nx[1]= {pe.value(M.x[1])}\nx[2]= {pe.value(M.x[2])}")

## Um Problema de Programação Convexa

$$\max{f(x)} = \ln{(x_1 + 1)} + x_2 $$
sujeito a:
$$2x_1 + x_2 \leq 3$$
$$x_1 \geq 0$$
$$x_2 \geq 0$$

In [ ]:
M = pe.ConcreteModel() 
M.J = pe.Set([1,2])
x = M.x = pe.Var(M.J, domain= pe.NonNegativeReals)
M.R = pe.Constraint(expr= 2*x[1] + 3*x[2] <= 3)
M.f = pe.Objective(expr=  pe.log(x[1] + 1) + x[2], sense= pe.maximize) 
pe.SolverFactory('ipopt').solve(M)

# Imprimindo o resultado na tela
print(f"f(x[1], x[2])= {pe.value(M.z)}\nx[1]= {pe.value(M.x[1])}\nx[2]= {pe.value(M.x[2])}")

## Um Problema de Programação Quadrática

$$\max{f(x)} = 15x_1 + 30x_2 + 4x_1x_2 - 2x_1^2 - 4x_2^2$$
sujeito a:
$$x_1 + 2x_2 \leq 30$$
$$x_1 \geq 0$$
$$x_2 \geq 0$$

In [ ]:
M = pe.ConcreteModel() 
M.J = pe.Set([1,2])
x = M.x = pe.Var(M.J, domain= pe.NonNegativeReals)
M.R = pe.Constraint(expr= x[1] + 2*x[2] <= 30)
M.f = pe.Objective(expr=  15*x[1] + 30*x[2] + 4*x[1]*x[2] - 2*x[1]**2 - 4*x[2]**2 , sense= pe.maximize) 
pe.SolverFactory('ipopt').solve(M)

# Imprimindo o resultado na tela
print(f"f(x[1], x[2])= {pe.value(M.z)}\nx[1]= {pe.value(M.x[1])}\nx[2]= {pe.value(M.x[2])}")

## Exercício 1 

$$\max{f(x_1, x_2)} = 5x_1 - x_1^2 + 8x_2 - 2x_2^2$$
sujeito a:
$$3x_1 + 2x_2 \leq 6$$
$$x_1 \geq 0$$
$$x_2 \geq 0$$

In [4]:
# Resolva aqui

## Exercício 2

$$\max{f(x_1, x_2)} = 24x_1 - x_1^2 + 10x_2 - x_2^2$$
sujeito a:
$$x_1 \leq 10$$
$$x_2 \leq 15$$
$$x_1 \geq 0$$
$$x_2 \geq 0$$

In [ ]:
# Resolva aqui